# Teleportation #

The methods for developing the circuits can be found in [src/algorithms/teleport.py](../src/algorithms/teleport.py).

## Description of test ##

- After creation of the entangled pair,
  a random state `|ψ⟩ = U|0⟩` is generated at start
  via a 'randomly generated' unitary `U`.
- To test that `|ψ⟩` is successfully teleported,
  after the teleportation protocol, we apply the inverse of `U`
  to Bob's state and measures it against `{|0⟩, |1⟩}`.
  The teleportation is thus successful exactly in case this bit is consistently `|0⟩`.
- We also expect that Alices bits be uniformly randomly distributed
  at the end of the teleportation protocol.

In the simulator, these expectations are indeed met.
In the backend, these expectations are only approximately attained.

In [ ]:
''' IMPORTS '''
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.quantum import *;
from src.demo.teleportation import *;

np.random.seed(39102901); # for repeatability

In [ ]:
action_display_circuit();

In [ ]:
'''Example with simulator'''
action_prepare_circuit_and_job(
    option = BACKEND_SIMULATOR.AER,
    num_samples = 100,
    num_shots = 100,
);

action_display_statistics(queue=False);

In [ ]:
'''Example with backend'''
action_prepare_circuit_and_job(
    option = BACKEND.LEAST_BUSY,
    num_samples = 10, # <- do not set this too high!
    num_shots = 100,
);

In [ ]:
'''Statistics for backend job - NOTE: job may be pending'''
action_display_statistics(queue=True);